# Segmentación de Clientes por Región y Categoría

Este notebook realiza la segmentación de clientes (o regiones/categorías como proxy) utilizando K-Means y PCA, basándose en datos de ventas, descuentos, márgenes y características temporales desde Snowflake. Los componentes principales incluyen:

- **Carga de datos** desde la vista `vw_ventas_ml`.
- **Preprocesamiento**: Agregación de datos por región y categoría, manejo de valores nulos, y escalado.
- **Reducción de dimensionalidad**: Uso de PCA para reducir la complejidad de los datos.
- **Clustering**: Aplicación de K-Means para identificar segmentos.
- **Visualización**: Gráficos interactivos con Plotly para mostrar los clústeres.
- **Almacenamiento**: Guardado de resultados en Snowflake.

**Objetivo**: Identificar grupos de clientes o combinaciones región-categoría con patrones de compra similares para estrategias personalizadas.

## 1. Importar Librerías y Configuración

In [1]:
import pandas as pd
import numpy as np
import snowflake.connector
import logging
from datetime import datetime
import os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Configurar advertencias y logging
warnings.filterwarnings("ignore")
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('customer_segmentation.log')
    ]
)

## 2. Conectar a la Base de Datos (Snowflake)

In [2]:
def get_snowflake_connection():
    try:
        conn = snowflake.connector.connect(
            user='TU_USUARIO',
            password='TU_CONTRASEÑA',
            account='TU_CUENTA',
            warehouse='TU_WAREHOUSE',
            database='BEBIDAS_PROJECT',
            schema='BEBIDAS_ANALYTICS'
        )
        logging.info("Conexión a Snowflake exitosa")
        return conn
    except Exception as e:
        logging.error(f"Error de conexión a Snowflake: {e}")
        raise

## 3. Cargar y Preprocesar Datos

In [3]:
def load_and_preprocess_data():
    try:
        conn = get_snowflake_connection()
        query = "SELECT * FROM vw_ventas_ml"
        df = pd.read_sql(query, conn)
        conn.close()
        if df.empty:
            raise ValueError("El DataFrame está vacío")
        logging.info(f"Datos cargados: {len(df)} registros")

        # Verificar las columnas disponibles
        logging.info(f"Columnas disponibles: {df.columns.tolist()}")

        # Detectar dinámicamente las columnas relevantes
        region_col = next((col for col in df.columns if 'region' in col.lower()), 'nombre_region')
        categoria_col = next((col for col in df.columns if 'categoria' in col.lower()), 'categoria')
        quantity_col = next((col for col in df.columns if any(keyword in col.lower() for keyword in ['cantidad', 'vendida', 'venta', 'unidades'])), 'CANTIDADES_VENDIDAS')
        discount_col = next((col for col in df.columns if 'desc' in col.lower()), 'DESC_PORCENTAJE')
        margin_col = next((col for col in df.columns if 'margen' in col.lower()), 'MARGEN_GANANCIA_BRUTA_PORCENTAJE')
        price_col = next((col for col in df.columns if 'precio' in col.lower()), 'PRECIO_PROMEDIO')
        logging.info(f"Columnas detectadas: region={region_col}, categoria={categoria_col}, cantidad={quantity_col}, descuento={discount_col}, margen={margin_col}, precio={price_col}")

        # Agregar datos por región y categoría (proxy para clientes)
        agg_df = df.groupby([region_col, categoria_col]).agg({
            quantity_col: 'mean',
            discount_col: 'mean',
            margin_col: 'mean',
            price_col: 'mean',
            'MES': 'nunique'  # Número de meses con datos
        }).reset_index()
        agg_df = agg_df.rename(columns={quantity_col: 'avg_quantity', discount_col: 'avg_discount', margin_col: 'avg_margin', price_col: 'avg_price', 'MES': 'months_active'})

        # Manejo de valores nulos
        agg_df[['avg_quantity', 'avg_discount', 'avg_margin', 'avg_price', 'months_active']] = agg_df[['avg_quantity', 'avg_discount', 'avg_margin', 'avg_price', 'months_active']].fillna(0)

        # Escalar los datos
        scaler = StandardScaler()
        features = ['avg_quantity', 'avg_discount', 'avg_margin', 'avg_price', 'months_active']
        X_scaled = scaler.fit_transform(agg_df[features])

        return agg_df, features, scaler, X_scaled, region_col, categoria_col
    except Exception as e:
        logging.error(f"Error en carga y preprocesamiento: {e}")
        raise

agg_df, features, scaler, X_scaled, region_col, categoria_col = load_and_preprocess_data()
print(f"Datos preprocesados. Filas: {len(agg_df)}, Features: {features}")

## 4. Reducción de Dimensionalidad con PCA

In [4]:
def apply_pca(X_scaled):
    try:
        pca = PCA(n_components=2)  # Reducir a 2 componentes para visualización
        X_pca = pca.fit_transform(X_scaled)
        explained_variance = pca.explained_variance_ratio_
        logging.info(f"Varianza explicada por componente: {explained_variance}")
        return X_pca, pca
    except Exception as e:
        logging.error(f"Error en PCA: {e}")
        raise

X_pca, pca = apply_pca(X_scaled)
agg_df = agg_df.assign(PC1=X_pca[:, 0], PC2=X_pca[:, 1])
print(f"PCA aplicada. Componentes: PC1, PC2")

## 5. Clustering con K-Means

In [5]:
def apply_kmeans(X_pca):
    try:
        # Probar diferentes valores de k con el método del codo
        inertia = []
        k_range = range(1, 11)
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=42)
            kmeans.fit(X_pca)
            inertia.append(kmeans.inertia_)

        # Seleccionar k óptimo (por ejemplo, k=3 basado en el codo)
        k_optimal = 3
        kmeans = KMeans(n_clusters=k_optimal, random_state=42)
        clusters = kmeans.fit_predict(X_pca)
        logging.info(f"Número óptimo de clústeres: {k_optimal}, Inercia: {kmeans.inertia_}")

        return clusters, kmeans
    except Exception as e:
        logging.error(f"Error en K-Means: {e}")
        raise

clusters, kmeans = apply_kmeans(X_pca)
agg_df = agg_df.assign(Cluster=clusters)
print(f"Clústeres asignados: {np.unique(clusters)}")

## 6. Visualización de Resultados

In [6]:
def plot_clusters(agg_df, region_col, categoria_col):
    try:
        fig = px.scatter(agg_df, x='PC1', y='PC2', color='Cluster',
                         hover_data=[region_col, categoria_col, 'avg_quantity', 'avg_discount'],
                         title='Segmentación de Clientes por Región y Categoría')
        fig.update_traces(marker=dict(size=12))
        fig.show()

        # Resumen por clúster
        cluster_summary = agg_df.groupby('Cluster').agg({
            'avg_quantity': 'mean',
            'avg_discount': 'mean',
            'avg_margin': 'mean',
            'avg_price': 'mean',
            'months_active': 'mean',
            region_col: lambda x: x.mode()[0],  # Región más común
            categoria_col: lambda x: x.mode()[0]  # Categoría más común
        }).reset_index()
        logging.info("Resumen por clúster:")
        logging.info(cluster_summary.to_string(index=False))
    except Exception as e:
        logging.error(f"Error en visualización: {e}")
        raise

plot_clusters(agg_df, region_col, categoria_col)

## 7. Almacenar Resultados

In [7]:
def save_results(agg_df):
    try:
        timestamp = datetime(2025, 6, 29, 9, 32)  # Fecha y hora actuales
        conn = get_snowflake_connection()
        cursor = conn.cursor()

        # Crear tabla en Snowflake
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Customer_Segmentation (
                id INTEGER AUTOINCREMENT START 1 INCREMENT 1,
                timestamp TIMESTAMP_NTZ,
                region VARCHAR(100),
                categoria VARCHAR(100),
                avg_quantity FLOAT,
                avg_discount FLOAT,
                avg_margin FLOAT,
                avg_price FLOAT,
                months_active INTEGER,
                cluster INTEGER,
                pc1 FLOAT,
                pc2 FLOAT
            )
        """)

        # Truncar tabla
        cursor.execute("TRUNCATE TABLE Customer_Segmentation")

        # Guardar resultados
        results_file = 'customer_segmentation.csv'
        agg_df[['region', 'categoria', 'avg_quantity', 'avg_discount', 'avg_margin', 'avg_price', 'months_active', 'Cluster', 'PC1', 'PC2']].to_csv(
            results_file, index=False, header=['region', 'categoria', 'avg_quantity', 'avg_discount', 'avg_margin', 'avg_price', 'months_active', 'cluster', 'pc1', 'pc2']
        )

        cursor.execute(
            f"PUT file://{os.path.abspath(results_file)} @%{conn.database}.BEBIDAS_ANALYTICS.Customer_Segmentation AUTO_COMPRESS=TRUE"
        )
        cursor.execute(
            f"COPY INTO Customer_Segmentation (timestamp, region, categoria, avg_quantity, avg_discount, avg_margin, avg_price, months_active, cluster, pc1, pc2)"
            f" FROM @%{conn.database}.BEBIDAS_ANALYTICS.Customer_Segmentation/{os.path.basename(results_file)}"
            f" FILE_FORMAT = (TYPE = 'CSV' SKIP_HEADER = 1 FIELD_DELIMITER = ',' TIMESTAMP_FORMAT = 'YYYY-MM-DD HH24:MI:SS')"
            f" ON_ERROR = 'CONTINUE'"
        )

        conn.commit()
        logging.info("Resultados de segmentación guardados exitosamente en Snowflake")

        cursor.execute("SELECT COUNT(*) FROM Customer_Segmentation")
        count = cursor.fetchone()[0]
        print(f"Registros en Customer_Segmentation: {count}")

    except Exception as e:
        conn.rollback()
        logging.error(f"Error al guardar resultados en Snowflake: {e}")
        raise
    finally:
        conn.close()

save_results(agg_df)